# Обычный резнет 18 с conv1d

In [39]:
import torch
import torch.nn as nn

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out

class ResNet18(nn.Module):
    def __init__(self, in_channels, num_classes=2):
        super(ResNet18, self).__init__()
        self.in_channels = in_channels

        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=2)

        self.in_channels = 64

        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2)
        self.layer3 = self._make_layer(256, 2)
        self.layer4 = self._make_layer(512, 2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels))
        self.in_channels = out_channels
        for _ in range(0, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        layers.append(nn.MaxPool1d(kernel_size=2))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        print(f'перед первым res блоком {x.shape}')

        x = self.layer1(x)
        print(f'после layer1 {x.shape}')
        x = self.layer2(x)
        print(f'после layer2 {x.shape}')
        x = self.layer3(x)
        print(f'после layer3 {x.shape}')
        x = self.layer4(x)
        print(f'после layer4 {x.shape}')

        x = self.avgpool(x)
        print(f'после avgpool {x.shape}')
        x = x.view(x.size(0), -1)
        print(f'после flattern {x.shape}')
        x = self.fc(x)

        return x

# Использование модели
input_channels = 12
signal_length = 2048
model = ResNet18(in_channels=input_channels)
input_data = torch.randn(1, input_channels, signal_length)
print(f'было на старте {input_data.shape}')
output = model(input_data)
print(f'на выходе {output.shape}')


было на старте torch.Size([1, 12, 2048])
перед первым res блоком torch.Size([1, 64, 1024])
после layer1 torch.Size([1, 64, 512])
после layer2 torch.Size([1, 128, 256])
после layer3 torch.Size([1, 256, 128])
после layer4 torch.Size([1, 512, 64])
после avgpool torch.Size([1, 512, 1])
после flattern torch.Size([1, 512])
на выходе torch.Size([1, 2])


# SE-Resnet18 (Squeeze-and-Excitation Networks)

Реализация из статьи \
Automatic Detection and Classification of 12-lead ECGs Using a Deep Neural Network\
Wenxiao Jia 1, Xian Xu 1, Xiao Xu 1, Yuyao Sun1, Xiaoshuang Liu1

In [40]:
import torch
import torch.nn as nn


class Squeeze_and_Excitation_block(nn.Module):
    def __init__(self, in_channels, reduction_ratio=2):
        super(Squeeze_and_Excitation_block, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc1 = nn.Linear(in_channels, in_channels//reduction_ratio)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(in_channels//reduction_ratio, in_channels)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        batch_size, num_channels, _ = x.size()
        residual = x
        print(f'  SE: на входе {residual.shape}')

        out = self.avgpool(x)
        print(f'  SE: после avgpool {out.shape}')
        
        out = out.view(batch_size, num_channels)
        print(f'  SE: view после avgpool {out.shape}')

        out = self.fc1(out)
        out = self.relu(out)
        print(f'  SE: fc1 выход {out.shape}')

        out = self.fc2(out)
        out = self.sigmoid(out)
        print(f'  SE: fc2 выход {out.shape}')

        out = out.view(batch_size, num_channels, 1)
        print(f'  SE: view после fc2 выход {out.shape}')

        out = residual * out.expand_as(x)
        print(f'  SE: результат {out.shape}')
        return out
    

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += self.shortcut(residual)
        out = self.relu(out)

        return out

class ResNet18(nn.Module):
    def __init__(self, in_channels, num_classes=2):
        super(ResNet18, self).__init__()
        self.in_channels = in_channels

        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=7, stride=1, padding=3, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool1d(kernel_size=2)

        self.in_channels = 64

        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2)
        self.layer3 = self._make_layer(256, 2)
        self.layer4 = self._make_layer(512, 2)

        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_channels, blocks):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels))
        self.in_channels = out_channels
        for _ in range(0, blocks):
            layers.append(BasicBlock(self.in_channels, out_channels))
        layers.append(nn.MaxPool1d(kernel_size=2))
        layers.append(Squeeze_and_Excitation_block(self.in_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        print(f'перед первым res блоком {x.shape}')

        x = self.layer1(x)
        print(f'после layer1 {x.shape}')
        x = self.layer2(x)
        print(f'после layer2 {x.shape}')
        x = self.layer3(x)
        print(f'после layer3 {x.shape}')
        x = self.layer4(x)
        print(f'после layer4 {x.shape}')

        x = self.avgpool(x)
        print(f'после avgpool {x.shape}')
        x = x.view(x.size(0), -1)
        print(f'после flattern {x.shape}')
        x = self.fc(x)

        return x

# Использование модели
input_channels = 12
signal_length = 2048
model = ResNet18(in_channels=input_channels)
input_data = torch.randn(1, input_channels, signal_length)
print(f'было на старте {input_data.shape}')
output = model(input_data)
print(f'на выходе {output.shape}')


было на старте torch.Size([1, 12, 2048])
перед первым res блоком torch.Size([1, 64, 1024])
  SE: на входе torch.Size([1, 64, 512])
  SE: после avgpool torch.Size([1, 64, 1])
  SE: view после avgpool torch.Size([1, 64])
  SE: fc1 выход torch.Size([1, 32])
  SE: fc2 выход torch.Size([1, 64])
  SE: view после fc2 выход torch.Size([1, 64, 1])
  SE: результат torch.Size([1, 64, 512])
после layer1 torch.Size([1, 64, 512])
  SE: на входе torch.Size([1, 128, 256])
  SE: после avgpool torch.Size([1, 128, 1])
  SE: view после avgpool torch.Size([1, 128])
  SE: fc1 выход torch.Size([1, 64])
  SE: fc2 выход torch.Size([1, 128])
  SE: view после fc2 выход torch.Size([1, 128, 1])
  SE: результат torch.Size([1, 128, 256])
после layer2 torch.Size([1, 128, 256])
  SE: на входе torch.Size([1, 256, 128])
  SE: после avgpool torch.Size([1, 256, 1])
  SE: view после avgpool torch.Size([1, 256])
  SE: fc1 выход torch.Size([1, 128])
  SE: fc2 выход torch.Size([1, 256])
  SE: view после fc2 выход torch.Size([